# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [3]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [4]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [5]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
Phragmen(marginal_slate, clear=True, redist=True)               1.0   
Phragmen(marginal_slate, clear=True, redist=False)              1.0   
Phragmen(marginal_slate, clear=False, redist=True)              1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
Phragmen(marginal_slate, clear=True, redist=True)                                     1.0   
Phragmen(marginal_slate, clear=True, redist=False)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=True)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...  

## Overall Performance

In [6]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               143.791667
ExactTotalUtilityMaximization(utility_transform=None)                                157.791667
LPTotalUtilityMaximization(utility_transform=None)                                   153.208333
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     145.458333
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      161.125000
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         154.166667
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    143.125000
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     154.833333
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        154.333333
Phragmen(marginal_slate, clear=True, redist=True)                                    144.875000
Phragmen(marginal_slate, clear=True, red

In [7]:
pd.DataFrame(np.trunc(df.values + 0/24+1*1e-6), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               128.0
ExactTotalUtilityMaximization(utility_transform=None)                                152.0
LPTotalUtilityMaximization(utility_transform=None)                                   150.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     134.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      151.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         152.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    130.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     144.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        152.0
Phragmen(marginal_slate, clear=True, redist=True)                                    125.0
Phragmen(marginal_slate, clear=True, redist=False)                                   128.0

## Differential Test Performance

In [8]:
non_uniform_columns = df.loc[:, df.nunique() > 1]
non_uniform_columns


vote                                                       Simple 3  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=None)         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
Phragmen(marginal_slate, clear=True, redist=True)          0.708333   
Phragmen(marginal_slate, clear=True, redist=False)         0.833333   
Phragmen(marginal_slate, clear=False, redist=True)         0.708333   
Phragmen(marginal_slate, clear=False, redist=Fa...         0.708333   
Phragmen(marginal_previous, clear=True, redist=...         0.833333   
Phragmen(marginal_previous, clear=True, redist=...         0.833333   
Phragmen(marginal_previous, clear=False, redist...         0.750000   
Phragmen(marginal_previous, clear=False, redist...         0.708333   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=None)                               1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                               1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                               1.000000   
Phragmen(marginal_slate, clear=True, redist=True)                                0.666667   
Phragmen(marginal_slate, clear=True, redist=False)                               0.875000   
Phragmen(marginal_slate, clear=False, redist=True)                               0.666667   
Phragmen(marginal_slate, clear=False, redist=Fa...                               0.791667   
Phragmen(marginal_previous, clear=True, redist=...                               0.833333   
Phragmen(marginal_previous, clear=True, redist=...                               0.666667   
Phragmen(marginal_previous, clear=False, redist...                               0.875000   
Phragmen(marginal_previous, clear=False, redist...                               0.666667   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=None)                                           1.000000    
GreedyTotalUtilityMaximization(utility_transfor...                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=Ge...                                           1.000000    
GreedyTotalUtilityMaximization(uti

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [9]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex 1.3             6
Ex A.1             6
Ex 3.1             6
Ex 4.2             6
Ex 4.3             5
Simple 3           5
Ex Alg1.5          5
Ex 1.1 modified    4
Ex C.2             4
Ex Alg A.1         4
Ex Alg A.2         4
Ex D.1             4
Ex Alg1.3          3
Ex 2.1             2
Ex B.3             2
Ex 4.1             1
Ex Alg1.4          1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            16
Maximum Coverage                                       14
Non-radical Minimum Utility Pareto Efficiency          14
Minimum Utility and Total Utility Pareto Efficiency    13
m-th Happiest Person Pareto Efficiency                  8
Individual Pareto Efficiency                            4
Name: count, dtype: int64


## Axiom Pass Rate

In [10]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_16248\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.000000,0.878086,0.868827,0.858025,0.725309,0.995370
ExactTotalUtilityMaximization(utility_transform=None),0.996914,0.996914,0.998457,1.000000,0.851852,1.000000
LPTotalUtilityMaximization(utility_transform=None),1.000000,0.959877,0.961420,0.962963,0.824074,0.966049
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.850309,0.853395,0.856481,0.836420,0.990741
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),0.998457,0.993827,0.993827,0.989198,0.993827,0.998457
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.944444,0.938272,0.942901,0.936728,0.947531
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),0.955247,0.876543,0.856481,0.820988,0.845679,0.945988
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),0.970679,0.961420,0.950617,0.952160,0.944444,0.955247
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.000000,0.941358,0.947531,0.935185,0.944444,0.947531
"Phragmen(marginal_slate, clear=True, redist=True)",0.962963,0.896605,0.905864,0.791667,0.873457,0.935185


# Debuggign

In [11]:
# alg = GreedyTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
alg = ExactTotalUtilityMaximization()
# alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))

In [12]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,,
Ex 1.1,1.000000,1.000000,1.000000,1.0,1.000000,1.0
Ex 1.1 modified,1.000000,1.000000,1.000000,1.0,0.083333,1.0
Ex 1.2,1.000000,1.000000,1.000000,1.0,1.000000,1.0
Ex 1.3,1.000000,1.000000,1.000000,1.0,1.000000,1.0
Ex 2.1,1.000000,1.000000,1.000000,1.0,1.000000,1.0
Ex 2.2,1.000000,1.000000,1.000000,1.0,1.000000,1.0
Ex 3.1,0.916667,0.916667,0.958333,1.0,1.000000,1.0
Ex 4.1,1.000000,1.000000,1.000000,1.0,0.458333,1.0
Ex 4.2,1.000000,1.000000,1.000000,1.0,1.000000,1.0


In [13]:
# case = rated_vote_cases["Ex 1.1 modified"]
from generative_social_choice.slates.voting_algorithms import RatedVoteCase
from generative_social_choice.slates.voting_utils import (
    CellWiseAugmentation, CandidateWiseAugmentation, VoterAndCellWiseAugmentation)


case = rated_vote_cases["Ex 3.1"]
case = RatedVoteCase(
    name="Ex 3.1_BIG",
    rated_votes=case.rated_votes,
    slate_size=case.slate_size,
    noise_augmentation_methods = {
                CellWiseAugmentation(): 40, 
                CandidateWiseAugmentation(): 100, 
                VoterAndCellWiseAugmentation(): 100
            }
)
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
# axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
# axiom = NonRadicalTotalUtilityAxiom()

case.rated_votes

,s1,s2,s3,s4,s5,s6
0,2,0,1,0,0,0
1,1,2,0,0,0,0
2,0,1,2,0,0,0
3,0,0,0,2,0,1
4,0,0,0,1,2,0
5,0,0,0,0,1,2


In [14]:
df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote,Ex 3.1,Ex 4.1,Ex 4.2,Ex 4.3,Ex 4.4,Ex B.1,Ex B.2
subtest,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency,Individual Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=None),0.916667,1.0,1.000000,1.0,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=None),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),0.958333,1.0,1.000000,1.0,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.000000,1.0,0.208333,1.0,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.000000,1.0,1.000000,1.0,1.0,1.0,1.0


In [15]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1', 's2', 's5']

In [16]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)

In [17]:
num_aug_cases = 240
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases[:num_aug_cases]):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [18]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
59,"[(s1, s2, s3), (s1, s2, s4), (s1, s2, s5), (s1...","[s1, s4, s6]",False
80,"[(s1, s2, s3), (s1, s2, s4), (s1, s2, s5), (s1...","[s3, s4, s6]",False
184,"[(s1, s2, s3), (s1, s2, s4), (s1, s2, s5), (s1...","[s2, s3, s4]",False
205,"[(s1, s2, s3), (s1, s2, s4), (s1, s2, s5), (s1...","[s1, s2, s5]",False


In [47]:
f0 = failures.iloc[3]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


,s1,s2,s3,s4,s5,s6
0,2.001070,0.001360,1.001134,0.001031,0.001313,0.001243
1,1.000895,2.000908,0.001010,0.000994,0.000782,0.000686
2,0.001394,1.001279,2.000949,0.001126,0.001296,0.001303
3,0.001155,0.001000,0.001106,2.000797,0.001160,1.000859
4,0.001229,0.001141,0.000985,1.001327,2.001202,0.001080
5,0.000920,0.000921,0.000957,0.001119,1.000977,2.000906


In [48]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4,s5,s6
0,1069.886650,1359.539118,1134.232812,1030.827708,1312.882703,1243.251633
1,894.784882,908.081365,1009.848250,993.814563,782.154441,686.419264
2,1394.100312,1278.624497,949.200265,1126.350413,1295.882314,1302.620719
3,1154.867382,1000.427458,1106.188919,797.453249,1160.234150,858.680535
4,1229.480306,1140.750950,984.634984,1326.931851,1201.691579,1080.198925
5,920.101349,921.356616,957.453441,1119.233707,976.641143,906.110865


In [49]:
f0.assignments

,candidate_id
0,s1
1,s2
2,s2
3,s1
4,s5
5,s5


In [50]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's2', 's5'],
   candidate_id
 0           s1
 1           s2
 2           s2
 3           s1
 4           s5
 5           s5)

In [ ]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

In [52]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['NonRadicalMinUtilityAxiom',
 'IndividualParetoAxiom',
 'CoverageAxiom',
 'NonRadicalTotalUtilityAxiom',
 'HappiestParetoAxiom',
 'MinimumAndTotalUtilityParetoAxiom']

# Scratch